In [1]:
import os
from math import sqrt

import torch
import torch.nn as nn
from torch_geometric.utils.loop import add_self_loops
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.nn import MessagePassing

from dig.xgraph.models.utils import subgraph
from dig.xgraph.evaluation import XCollector

import import_ipynb
from influenceDataset import get_dataloader, influenceDataset
from gnnNets import get_gnnNets
from base_explainer import ExplainerBase

importing Jupyter notebook from influenceDataset.ipynb
importing Jupyter notebook from gnnNets.ipynb
importing Jupyter notebook from base_explainer.ipynb


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3" 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
"""
왜 self loop를 추가하는지 모르겠다.
"""
class RandomSelectorExplainer(ExplainerBase):
    def __init__(self, model: nn.Module, explain_graph: bool = False):
        super().__init__(model=model, explain_graph=explain_graph)        
        
        
    def forward(self, x, edge_index, edge_attr, **kwargs):
        super().forward(x, edge_index, edge_attr)
        self.model.eval()

        # Assume the mask we will predict
        labels = tuple(i for i in range(kwargs.get('num_classes')))
        ex_labels = tuple(torch.tensor([label]).to(self.device) for label in labels)

        if self.explain_graph:
            self_loop_edge_index, _ = add_self_loops(edge_index, num_nodes=x.shape[0])
            edge_mask = torch.rand(self_loop_edge_index.shape[1])
            edge_masks = [edge_mask for _ in ex_labels]

            self.__clear_masks__()
            self.__set_masks__(x, self_loop_edge_index)
            hard_edge_masks = [self.control_sparsity(edge_mask, sparsity=kwargs.get('sparsity')).sigmoid().to(self.device)
                               for _ in ex_labels]
            

            with torch.no_grad():
                related_preds = self.eval_related_pred(
                    x, edge_index, edge_attr, hard_edge_masks)
            self.__clear_masks__()

        """
        else:
            node_idx = kwargs.get('node_idx')
            if not node_idx.dim():
                node_idx = node_idx.reshape(-1)
            node_idx = node_idx.to(self.device)
            assert node_idx is not None

            self_loop_edge_index, _ = add_self_loops(edge_index, num_nodes=x.shape[0])

            _, _, _, self.hard_edge_mask = subgraph(
                node_idx, self.__num_hops__, self_loop_edge_index,
                relabel_nodes=True, num_nodes=None, flow=self.__flow__())

            edge_mask = torch.rand(self_loop_edge_index.shape[1])

            self.__clear_masks__()
            self.__set_masks__(x, self_loop_edge_index)
            edge_masks = [edge_mask for _ in ex_labels]
            hard_edge_masks = [self.control_sparsity(
                edge_mask, sparsity=kwargs.get('sparsity')).sigmoid().to(self.device) for _ in ex_labels]
            
            
            with torch.no_grad():
                related_preds = self.eval_related_pred(
                    x, edge_index, hard_edge_masks, **kwargs)
            self.__clear_masks__()
        """

        return edge_masks, hard_edge_masks, related_preds

In [9]:
def pipeline():
    dataset = influenceDataset('/data/URP','/data/URP/graphs')
        
    model = get_gnnNets(1, 1, {'gnn_latent_dim':[128,128,128]})
    model.load_state_dict(torch.load('/data/URP/model/bestmodel.pt'))
    model.to(device)
    
    random_explainer = RandomSelectorExplainer(model, explain_graph=True)
    
    index = 0
    x_collector = XCollector()
    for i, data in enumerate(dataset):
        index += 1
        data.to(device)
        walks, masks, related_preds = \
            random_explainer(data.x, data.edge_index, data.edge_attr,
                             sparsity=0.1,
                             num_classes=dataset.num_classes)
        
        model.eval()
        prediction = model(data).argmax(-1).item()
        x_collector.collect_data(masks, related_preds, label=prediction)
    
    print(f'Fidelity: {x_collector.fidelity:.4f}\n'
          f'Fidelity_inv: {x_collector.fidelity_inv: .4f}\n'
          f'Sparsity: {x_collector.sparsity:.4f}')

In [10]:
pipeline()

Fidelity: 3.9890
Fidelity_inv:  0.6591
Sparsity: 0.1000
